In [1]:
#### Written and Copyright by Mohit Agarwal
#### Georgia Institute of Technology
#### Email: magarwal37@gatech.edu

### Code to decode Error-Potentials for the data recording on any game
### Data should be present in the format of SXX-MMDD-XX.csv, _labels, and _metadata
### 16 Electrode Channels: desribed in metadata file e.g. ['Pz', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'Cz', 'Fz', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'Fpz']

### If want to classify data for every subject individually - put separate folder for every subject
### If training and testing on different files, the folder structure should be similar in both train and test

In [2]:
##Importing Libraries

import numpy as np
import pandas as pd
import bisect
import os
import math
from scipy.signal import *
from pylab import *
from collections import Counter
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import ElasticNet, LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from random import shuffle
from sklearn import svm
from sklearn import metrics
import mne
from mne.time_frequency import psd_array_multitaper, psd_multitaper
import matplotlib
import matplotlib.pyplot as plt
import pyriemann
from tabulate import tabulate

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

mne.set_log_level('ERROR')
np.set_printoptions(precision=3)

/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
## Fixed Parameters of the Algorithm
data_dir = 'data'
train_dir = 'train'
test_dir = 'test'
filetype = 'openvibe'
channels = ['Pz', 'F4', 'C4', 'P4', 'O2', 'F8', 'Fp2', 'Cz', 'Fz', 'F3', 'C3', 'P3', 'O1', 'F7', 'Fp1', 'Fpz']

stim_nonErrp = 1
stim_Errp = 0
stim_dontknow = -1

th_1 = 75 if filetype=='openvibe' else 150 # in uV : for adjacent spikes
th_2 = 150 if filetype=='openvibe' else 300 # in uV : for min-max
freq = 125.0
time_delay = 0.0 # in seconds

In [4]:
## Parameters of the algorithm [to change]

# Misc
flag_test = False # True if testing o.w. Cross-validate
plt_err_dist = False  # Plots Error Distribution in time if set True
selected_channels = None#[0,1,2,3,7,8,9,10,11,15] # If empty means all channels are selected, other option: [0,4,5]
sync_method = "event_date" # options are "default" and "event_date"
balanced_sampling = "oversample"



# Plot-based
plot_GAERP_allchan = False
plot_GAERP_onechan = False
plot_ERP_onechan = False
chan_toPlot = 4

total_channels = 16 if selected_channels is None else len(selected_channels) 
if selected_channels:
    channels = [channels[i] for i in selected_channels]

prob_th = 0.7
num_sims = 10 # only used for cross-fold validation

# Classification based overall
low_freq = 1.0
baseline_epoc = int(0.2*freq)
epoc_window = int(0.8*freq)
butter_filt_order = 4
cv_folds = 10
mean_baseline = False
mean_per_chan = False
car_pre = False #True
high_freq = 15.0
car_post = False


xdawn_filters_spatial = 4
xdawn_filters_time = 4
xdawn_filters_freq = 4
mean_baseline_spatial = True
mean_baseline_time = True
mean_baseline_freq = False
mean_baseline_freq_pre = True
nbins = 16

kernel_freq, kernel_time = 'linear', 'linear'
C_freq, C_time = 1.0, 1.0
tmin_freq, tmax_freq = 0.4, 1.0
fmin_freq, fmax_freq = 1, 14

all_models = ['ensemble','spatial','freq','time']
classifier = 'ensemble' # other options - 'spatial', 'freq', 'time', 'ensemble'

In [5]:
## Helper Functions

# function to convert missing values to 0
convert = lambda x: float(x.strip() or float('NaN'))

def convert_codes(x):
    if ':' in x:
        return x.split(':',1)[1]
    else:
        return (x.strip() or float('NaN'))
    
def convert_time(t):
    if '->' in t:
        t, _ = t.split('->')
        
    (t1, ms) = t.split('.')
    (h, m, s) = t1.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) + 0.001*int(ms)

# function to convert stimulations
def to_byte(value, length):
    for x in range(length):
        yield value%256
        value//=256

# function to bandpass filter
def bandpass(sig,band,fs,butter_filt_order):
    B,A = butter(butter_filt_order, np.array(band)/(fs/2), btype='bandpass')
    return lfilter(B, A, sig, axis=0)

# Function to initialize confusion matrices
def init_confusion_matrix():
    cmat_train, cmat_test, auc_test = {}, {}, {}
    if classifier=='ensemble':
        for model in all_models:
            cmat_train[model], cmat_test[model], auc_test[model] = np.zeros([3,3]), np.zeros([3,3]), []
    else:
        cmat_train[classifier], cmat_test[classifier], auc_test[classifier] = np.zeros([3,3]), np.zeros([3,3]), []
        
    return cmat_train, cmat_test, auc_test

In [6]:
def loadData(DataFolder, label_file):
    raw_file = label_file.split('_')[0] + '.csv'
    if filetype=='openvibe':
        raw_EEG = np.loadtxt(open(os.path.join(DataFolder,raw_file), "rb"), delimiter=",", skiprows=1, usecols=(0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17))
    elif filetype=='openbci':
        raw_EEG = np.loadtxt(open(os.path.join(DataFolder,raw_file), "rb"), delimiter=",", skiprows=6, usecols=(22,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16), converters={22: convert_time})
        exp_start_time = raw_EEG[0,0]
        raw_EEG[:,0] = raw_EEG[:,0] - exp_start_time
    stim = pd.read_csv(os.path.join(DataFolder,label_file))
    return raw_EEG, stim

def pre_process(raw_EEG):
    # Step 1: Subtract Mean per electrode
    sig = raw_EEG[:,1:]
    sig = sig[:,selected_channels] if selected_channels is not None else sig
    if mean_per_chan:
        sig_mean = np.mean(sig, axis=0)
        sig = sig - sig_mean
        
    # Step 2: Bandpass in the given frequency range
    sigF = bandpass(sig, [low_freq,high_freq], freq, butter_filt_order)
    
    # Step 3: Removing Average of all channels
    if car_pre:
        sig_mean = np.mean(sigF, axis=1)
        sigF = sigF - np.reshape(sig_mean, [len(sig_mean),1])*np.ones([1,total_channels])
    
    return sigF

def post_process(X):
    if car_post:
        temp =  np.repeat(np.mean(X,axis=2),X.shape[2],axis=1)
        temp = np.reshape(temp, X.shape)
        X = X - temp
    X = X.transpose((0,2,1))
    return X
    

# Arrange EEG data for training/testing
def arrange_data(EEG, sigF, stim):
    X = []
    Y = []
    for stim_id, stim_code in enumerate(stim['label']):
        if (not math.isnan(stim_code)) and stim_code!=33552 and stim_code!=33553 and stim_code!=33554:
            if stim_code==stim_nonErrp or stim_code==stim_Errp:
                if sync_method == "default":
                    time_instant = stim['time1'][stim_id] + time_delay
                else:
                    time_instant = stim['time2'][stim_id] + time_delay
                time_instant = time_instant
                idx = bisect.bisect(EEG[:,0],time_instant)
                X_temp = sigF[idx-1-baseline_epoc:idx+epoc_window,:]
                if mean_baseline:
                    X_mean = np.mean(sigF[idx-baseline_epoc:idx,:],0)
                    X_temp = X_temp - X_mean
                check = False
                # Removing nan values and corrupted data
                if np.isnan(X_temp).any():
                    check = True
                for i in range(total_channels):  
                    X_diff = [abs(t - s) for s, t in zip(X_temp[:,i], X_temp[1:,i])]
                    if max(X_diff)>th_1 or max(X_temp[:,i])-min(X_temp[:,i])>th_2:
                        check = True
                if not check:
                    X.append(X_temp)
                    Y.append(stim_code)
    return np.array(X), np.array(Y)

In [7]:
# Pre-processing Data from Train/Test Directory

def dir_list(folder_path):
    dir_list = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    dir_list.append('')
    return dir_list

data_dict = {}
train_folder = os.path.join(data_dir,train_dir)
test_folder = os.path.join(data_dir,test_dir)
list_of_dir = dir_list(train_folder)
if flag_test:
    assert list_of_dir==dir_list(test_folder), "Error: Test directory does not have same structure as train"
total_dirs = len(list_of_dir)
for dir_idx in range(total_dirs):
    paths = [["train", os.path.join(train_folder, list_of_dir[dir_idx])], ["test", os.path.join(test_folder, list_of_dir[dir_idx])]]
    if not flag_test:
        paths.pop(1)
    for items in paths:
        train_test = items[0]
        folder = items[1]
        list_of_files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and '_labels' in f]
#         print list_of_files
        for curr_file in list_of_files:
            
            # Step 1: Load Raw EEG data (with filtered stimulations)
            EEG, stim = loadData(folder, curr_file)
            
            # Step 2: Pre-process the raw EEG data before cutting in time-windows
            sig = pre_process(EEG)
            
            # Step 3: Arrange Data
            X_curr, Y_curr = arrange_data(EEG, sig, stim)
            
            # Making sure either the stimulations are present or raw data during stimulation is not discarded
            if Y_curr.shape[0] > 0:
                # Step 4: Post-process the Signals
                X_curr = post_process(X_curr)

                # Step 5: Arrange data in dictionary
                key = 'data_' + list_of_dir[dir_idx] + '$' + train_test
                if key in data_dict:
                    [X, Y] = data_dict[key]
                    X = np.concatenate((X,X_curr),axis=0)
                    Y = np.concatenate((Y,Y_curr),axis=0)
                    data_dict[key] = [X, Y]
                else:
                    data_dict[key] = [X_curr, Y_curr]
                
            
exps = set()
for key in data_dict.keys():
    exps.add(key.split("$")[0])

# elements in the 'exps' set holds the folder names
# [X_train, Y_train] = data_dict[exp+"$train"] loads the X_train and Y_train for training samples
# All samples in all files inside one folder are concatenated when storing in data_dict

# Size of X = [#of stim x #chan x # of timestamps]

In [8]:
# Different Classification Pipelines after pre-processing

@ignore_warnings(category=ConvergenceWarning)
def pipeline_spatial(X_train, X_test, Y_train):
    
    X_train_baseline_mean = np.mean(X_train[:,:,:baseline_epoc],2)
    X_test_baseline_mean = np.mean(X_test[:,:,:baseline_epoc],2)

    X_train = X_train[:,:,baseline_epoc:]
    X_test = X_test[:,:,baseline_epoc:]
    if mean_baseline_spatial:
        X_train = X_train - np.repeat(X_train_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
        X_test = X_test -  np.repeat(X_test_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
    
    filt_1 = pyriemann.estimation.XdawnCovariances(nfilter=xdawn_filters_spatial, estimator='lwf', xdawn_estimator='lwf')
    filt_2 = pyriemann.channelselection.ElectrodeSelection(nelec=8, metric='riemann')
    filt_3 = pyriemann.tangentspace.TangentSpace(metric='riemann', tsupdate=False) # 36-sized vector for nelec=8
    filt_4 = Normalizer(norm='l1')                       
    filt_5 = ElasticNet(l1_ratio=0.05, alpha=0.02, normalize=True)
    filt_5 = SGDClassifier(loss='squared_hinge', class_weight='balanced', penalty='elasticnet', l1_ratio=0.5, alpha=0.002, max_iter=1000, tol=0.001)
    
    clf = make_pipeline(filt_1, filt_2, filt_3, filt_4)
#     clf = make_pipeline(filt_1, filt_2, filt_3, filt_4, filt_5)
    clf.fit(X_train, Y_train)
    train_features = clf.transform(X_train)
    test_features = clf.transform(X_test)

#     train_scores = clf.predict(X_train)
#     test_scores = clf.predict(X_test)

    
#     thresholds = [x/100.0 for x in range(20,81,1)]
#     acc_tr = []
#     for th in thresholds:
#         acc_tr.append([np.mean((train_scores > th)==Y_train),th])
#     decision_th = max(acc_tr)[1]
#     max_val = max(train_scores) - decision_th
#     min_val = decision_th - min(train_scores)
#     train_proba, test_proba = [], []
    
#     for item in train_scores:
#         prob = [None, None]
#         if item < decision_th:
#             prob[0] = min(1, (decision_th - item)/(min_val))
#             prob[1] = 1-prob[0]
#         else:
#             prob[1] = min(1, (item - decision_th)/(max_val))
#             prob[0] = 1-prob[1]
#         train_proba.append(prob)
#     for item in test_scores:
#         prob = [None, None]
#         if item < decision_th:
#             prob[0] = min(1, (decision_th - item)/(min_val))
#             prob[1] = 1-prob[0]
#         else:
#             prob[1] = min(1, (item - decision_th)/(max_val))
#             prob[0] = 1-prob[1]
#         test_proba.append(prob)
    
    
    clf_isotonic = CalibratedClassifierCV(filt_5, cv=2, method='isotonic')
    clf_isotonic.fit(train_features, Y_train)
    train_scores = clf_isotonic.predict_proba(train_features)
    test_scores = clf_isotonic.predict_proba(test_features)
    
#     print(train_scores.shape, test_scores.shape)
                                                           
    return train_scores, test_scores #np.array(train_proba), np.array(test_proba)

def pipeline_time(X_train, X_test, Y_train):
    
    X_train_baseline_mean = np.mean(X_train[:,:,:baseline_epoc],2)
    X_test_baseline_mean = np.mean(X_test[:,:,:baseline_epoc],2)
    
#     B,A = butter(butter_filt_order, np.array([1.0, 10.0])/(freq/2), btype='bandpass')
    
#     X_train = lfilter(B, A, X_train, axis=2)
#     X_test = lfilter(B, A, X_test, axis=2)

    X_train = X_train[:,:,baseline_epoc:]
    X_test = X_test[:,:,baseline_epoc:]
    if mean_baseline_time:
#         X_train_baseline_mean = np.reshape(X_train_baseline_mean, [,1]
        X_train = X_train - np.repeat(X_train_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
        X_test = X_test -  np.repeat(X_test_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
        
    filt_1 = pyriemann.spatialfilters.Xdawn(nfilter=xdawn_filters_time, estimator='lwf')
    X_train = filt_1.fit_transform(X_train, Y_train)
    X_test = filt_1.transform(X_test)
    
    t_idxs = int(epoc_window/nbins)
    X_train_time = np.zeros([X_train.shape[0], xdawn_filters_time*2, nbins]) 
    for idd in range(X_train.shape[0]):
        for i_bin in range(nbins):
            t_range_start, t_range_end = i_bin*t_idxs, (i_bin+1)*t_idxs -1
            X_train_time[idd,:,i_bin] = np.mean(X_train[idd,:,t_range_start:t_range_end], 1)
            
    X_train_time = X_train_time.reshape([X_train_time.shape[0], X_train_time.shape[1]*X_train_time.shape[2]])
    
    
    X_test_time = np.zeros([X_test.shape[0], xdawn_filters_time*2, nbins]) 
    for idd in range(X_test.shape[0]):
        for i_bin in range(nbins):
            t_range_start, t_range_end = i_bin*t_idxs, (i_bin+1)*t_idxs -1
            X_test_time[idd,:,i_bin] = np.mean(X_test[idd,:,t_range_start:t_range_end], 1)
            
    X_test_time = X_test_time.reshape([X_test_time.shape[0], X_test_time.shape[1]*X_test_time.shape[2]])
    
    filt_4 = Normalizer(norm='l2')  
    filt_5 = svm.SVC(kernel=kernel_time, C=C_time, class_weight='balanced', gamma='auto', probability=True)
    clf  = make_pipeline(filt_4, filt_5)
    clf.fit(X_train_time, Y_train)
    train_scores = clf.predict_proba(X_train_time)
    
    test_scores = clf.predict_proba(X_test_time)
    return train_scores, test_scores
            

def pipeline_freq(X_train, X_test, Y_train):
    
    X_train_baseline_mean = np.mean(X_train[:,:,:baseline_epoc],2)
    X_test_baseline_mean = np.mean(X_test[:,:,:baseline_epoc],2)
    if mean_baseline_freq_pre:
#         X_train_baseline_mean = np.reshape(X_train_baseline_mean, [,1]
        X_train = X_train - np.repeat(X_train_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
        X_test = X_test -  np.repeat(X_test_baseline_mean[:,:,np.newaxis], X_train.shape[2], axis=2)
        
    filt_1 = pyriemann.spatialfilters.Xdawn(nfilter=xdawn_filters_time, estimator='lwf')
    X_train = filt_1.fit_transform(X_train, Y_train)
    X_test = filt_1.transform(X_test)
    
    filt_1 = pyriemann.spatialfilters.Xdawn(nfilter=xdawn_filters_freq, estimator='lwf')
    X_train = filt_1.fit_transform(X_train, Y_train)
    X_test = filt_1.transform(X_test)
    info = mne.create_info(ch_names=2*xdawn_filters_freq*['a'], sfreq=freq, ch_types=2*xdawn_filters_freq*['eeg'])
    
    # 14 for 2-26, 0.6s long
    # 7 for 1-12, 0.6s long
    X_train_freq = np.zeros([X_train.shape[0], 2*xdawn_filters_freq, 8])    
    for idd in range(X_train.shape[0]):
        raw = mne.io.RawArray(X_train[idd,:,:], info)
        psds, freqs = psd_multitaper(raw, low_bias=False, tmin=tmin_freq, tmax=tmax_freq, fmin=fmin_freq, fmax=fmax_freq, proj=False, n_jobs=8)
        if mean_baseline_freq:
            psds_base, freqs_base = psd_multitaper(raw, low_bias=False, tmin=0.0, tmax=0.2, fmin=fmin_freq, fmax=fmax_freq, proj=False, n_jobs=8)
            temp = 10*np.log10(psds) - np.mean(psds_base)
        else:
            temp = 10*np.log10(psds)
        X_train_freq[idd,:,:] = temp[:,:]
        
        
    X_train_freq = X_train_freq.reshape([X_train_freq.shape[0], X_train_freq.shape[1]*X_train_freq.shape[2]])
    
    X_test_freq = np.zeros([X_test.shape[0], 2*xdawn_filters_freq, 8])    
    for idd in range(X_test.shape[0]):
        raw = mne.io.RawArray(X_test[idd,:,:], info)
        psds, freqs = psd_multitaper(raw, low_bias=False, tmin=tmin_freq, tmax=tmax_freq, fmin=fmin_freq, fmax=fmax_freq, proj=False, n_jobs=8)
        if mean_baseline_freq:
            psds_base, freqs_base = psd_multitaper(raw, low_bias=False, tmin=0.0, tmax=0.2, fmin=fmin_freq, fmax=fmax_freq, proj=False, n_jobs=8)
            temp = 10*np.log10(psds) - np.mean(psds_base)
        else:
            temp = 10*np.log10(psds)
        X_test_freq[idd,:,:] = temp[:,:]
    X_test_freq = X_test_freq.reshape([X_test_freq.shape[0], X_test_freq.shape[1]*X_test_freq.shape[2]])
    
#     filt_4 = Normalizer(norm='l2')  
    filt_5 = svm.SVC(kernel=kernel_freq, C=C_freq, class_weight='balanced', gamma='auto', probability=True)
    clf  = make_pipeline(filt_5)
    clf.fit(X_train_freq, Y_train)
    train_scores = clf.predict_proba(X_train_freq)
    
    test_scores = clf.predict_proba(X_test_freq)
    return train_scores, test_scores
    
    



In [9]:
def balance_dataset(X_train, Y_train): 
    if balanced_sampling == "undersample":
        rusU = RandomUnderSampler(return_indices=True)
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resU, Y_resU, id_rusU = rusU.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resU,[X_resU.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resU
    elif balanced_sampling == "oversample":
        rusO = RandomOverSampler(return_indices=True)
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resO, Y_resO, id_rusO = rusO.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resO,[X_resO.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resO
    elif balanced_sampling == "smote":
        rusS = SMOTE()
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resS, Y_resS = rusS.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resS,[X_resS.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resS
        
    return X_sim, Y_sim

In [10]:
# Acutal Prediction

def actual_prediction(train_prob, test_prob):
    train_preds, test_preds = np.zeros([train_prob.shape[0]]), np.zeros([test_prob.shape[0]])
    for idx in range(train_preds.shape[0]):
#         if train_prob[idx] >= 0.6:
#             train_preds[idx] = 1
#         else:
#             train_preds[idx] = 0
#     for idx in range(test_preds.shape[0]):
#         if test_prob[idx] >= 0.55:
#             test_preds[idx] = 1
#         else:
#             test_preds[idx] = 0
        if train_prob[idx] >= prob_th:
            train_preds[idx] = 1
        elif train_prob[idx] < 1-prob_th:
            train_preds[idx] = 0
        else:
            train_preds[idx] = -1
    for idx in range(test_preds.shape[0]):
        if test_prob[idx] >= prob_th:
            test_preds[idx] = 1
        elif test_prob[idx] < 1-prob_th:
            test_preds[idx] = 0
        else:
            test_preds[idx] = -1
    
    return train_preds, test_preds

def predict(X_train, X_test, Y_train, Y_test):
    train_preds, test_preds = [], []
    cmat_train, cmat_test, auc_test = dict(), dict(), dict()
    
    if classifier=='spatial':
        train_spatial_prob, test_spatial_prob = pipeline_spatial(X_train, X_test, Y_train)
        train_preds, test_preds = actual_prediction(train_spatial_prob[:,1], test_spatial_prob[:,1])
        cmat_train['spatial'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['spatial'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])  
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_spatial_prob, pos_label=1) #label=0 corresponds to errp
        auc_test['spatial'] = metrics.auc(fpr,tpr)
    elif classifier=='freq':
        train_freq_prob, test_freq_prob = pipeline_freq(X_train, X_test, Y_train)
        train_preds, test_preds = actual_prediction(train_freq_prob[:,1], test_freq_prob[:,1])
        cmat_train['freq'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['freq'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])  
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_freq_prob, pos_label=1) #label=0 corresponds to errp
        auc_test['freq'] = metrics.auc(fpr,tpr)
    elif classifier=='time':
        train_time_prob, test_time_prob = pipeline_time(X_train, X_test, Y_train)
        train_preds, test_preds = actual_prediction(train_time_prob[:,1], test_time_prob[:,1])
        cmat_train['time'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['time'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])  
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_time_prob, pos_label=1) #label=0 corresponds to errp
        auc_test['time'] = metrics.auc(fpr,tpr)
    elif classifier=='ensemble': # classifier is ensemble
        train_spatial_prob, test_spatial_prob = pipeline_spatial(X_train, X_test, Y_train)
        train_freq_prob, test_freq_prob = pipeline_freq(X_train, X_test, Y_train)
        train_time_prob, test_time_prob = pipeline_time(X_train, X_test, Y_train)
        
        train_prob = (train_spatial_prob[:,1] + train_time_prob[:,1] + train_freq_prob[:,1])/3.0
        test_prob = (test_spatial_prob[:,1] + test_time_prob[:,1] + test_freq_prob[:,1])/3.0
        #Spatial
        train_preds, test_preds = actual_prediction(train_spatial_prob[:,1], test_spatial_prob[:,1])
        cmat_train['spatial'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['spatial'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1]) 
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_spatial_prob[:,1], pos_label=1) #label=0 corresponds to errp
        auc_test['spatial'] = metrics.auc(fpr,tpr)
        #Freq
        train_preds, test_preds = actual_prediction(train_freq_prob[:,1], test_freq_prob[:,1])
        cmat_train['freq'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['freq'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])  
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_freq_prob[:,1], pos_label=1) #label=0 corresponds to errp
        auc_test['freq'] = metrics.auc(fpr,tpr)
        #Time
        train_preds, test_preds = actual_prediction(train_time_prob[:,1], test_time_prob[:,1])
        cmat_train['time'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['time'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_time_prob[:,1], pos_label=1) #label=0 corresponds to errp
        auc_test['time'] = metrics.auc(fpr,tpr)
        #Ensemble
        train_preds, test_preds = actual_prediction(train_prob, test_prob)
        cmat_train['ensemble'] = confusion_matrix(Y_train, train_preds, labels=[-1,0,1])
        cmat_test['ensemble'] = confusion_matrix(Y_test, test_preds, labels=[-1,0,1])
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, test_prob, pos_label=1) #label=0 corresponds to errp
        auc_test['ensemble'] = metrics.auc(fpr,tpr)
        
#         for idx in range(test_preds.shape[0]):
#             if not (test_preds[idx] == Y_test[idx]):
#                 print test_preds[idx], Y_test[idx], test_spatial_prob[idx,1], test_freq_prob[idx,1], test_time_prob[idx,1], test_prob[idx]
                
        
        

    # here index 1 and value 1 - corresponds to the Non-ErrP
    

    return cmat_train, cmat_test, auc_test, test_preds

In [11]:
# Cross Validation: Returns train and test confusion matrix for X and Y

def predict_CV(X, Y):    
    kfold = KFold(n_splits=cv_folds, shuffle=True)
    cmat_train, cmat_test, auc_test = init_confusion_matrix()
        
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
      
        X_train, Y_train = balance_dataset(X_train, Y_train)
        
        cm_train, cm_test, auc_curr, _ = predict(X_train, X_test, Y_train, Y_test)
        
        if classifier=='ensemble':
            for model in all_models:
                cmat_train[model] = cmat_train[model] + cm_train[model]
                cmat_test[model] = cmat_test[model] + cm_test[model]
#                 auc_test[model] = auc_test[model] + auc_curr[model]
                auc_test[model].append(auc_curr[model])
        else:
            cmat_train[classifier] = cmat_train[classifier] + cm_train[classifier]
            cmat_test[classifier] = cmat_test[classifier] + cm_test[classifier]
#             auc_test[classifier] = auc_test[classifier] + auc_curr[classifier]
            auc_test[classifier].append(auc_curr[classifier])
    
    if classifier=='ensemble':
        for model in all_models:
            cmat_train[model] = cmat_train[model]*1.0/cv_folds
            cmat_test[model] = cmat_test[model]*1.0/cv_folds
#             auc_test[model] = auc_test[model]*1.0/cv_folds
            auc_test[model] = mean(auc_test[model])
    else:
        cmat_train[classifier] = cmat_train[classifier]*1.0/cv_folds
        cmat_test[classifier] = cmat_test[classifier]*1.0/cv_folds
#         auc_test[classifier] = auc_test[classifier]*1.0/cv_folds
        auc_test[classifier] = mean(auc_test[classifier])
            
    return cmat_train, cmat_test, auc_test

In [12]:
# Computing Accuracies

result_dict = {}
    
cmat_train, cmat_test, auc_test = init_confusion_matrix()

#ToDO: FIX flag_test part of code
for exp_idx, exp in enumerate(exps):
    print("Processing.. "+str(exp_idx)+"/"+str(len(exps))+"")
    Y_pred = None
    Y_test = None
    if flag_test:
        [X_train, Y_train] = data_dict[exp+"$train"]
        index_shuffle = [i for i in range(X_train.shape[0])]
        shuffle(index_shuffle)
        X_train = X_train[index_shuffle,:,:]
        Y_train = Y_train[index_shuffle]
        [X_test, Y_test] = data_dict[exp+"$test"]
        X_train, Y_train = balance_dataset(X_train, Y_train)
        cmat_train, cmat_test, auc_test, Y_pred = predict(X_train, X_test, Y_train, Y_test)
    else:
        [X_train, Y_train] = data_dict[exp+"$train"]
        
        cmat_train, cmat_test, auc_test = init_confusion_matrix()
    
        for ind in range(num_sims):
            cm_train, cm_test, auc_curr = predict_CV(X_train,Y_train)
            
            if classifier=='ensemble':
                for model in all_models:
                    cmat_train[model] = cmat_train[model] + cm_train[model]
                    cmat_test[model] = cmat_test[model] + cm_test[model]
#                     auc_test[model] = auc_test[model] + auc_curr[model]
                    auc_test[model].append(auc_curr[model])
            else:
                cmat_train[classifier] = cmat_train[classifier] + cm_train[classifier]
                cmat_test[classifier] = cmat_test[classifier] + cm_test[classifier]
#                 auc_test[classifier] = auc_test[classifier] + auc_curr[classifier]
                auc_test[classifier].append(auc_curr[classifier])
        
        for model in cmat_train.keys():
            cmat_train[model] = cmat_train[model]/num_sims
            cmat_test[model] = cmat_test[model]/num_sims
#             auc_test[model] = auc_test[model]/num_sims
            auc_test[model] = mean(auc_test[model])
    
    assert not exp in result_dict.keys()
    train_errp_split = (sum(cmat_train[classifier],1)*1.0/sum(cmat_train[classifier]))[1]
    test_errp_split = (sum(cmat_test[classifier],1)*1.0/sum(cmat_test[classifier]))[1]
    result_dict[exp] = [cmat_train, cmat_test, auc_test, X_train.shape[0], train_errp_split, test_errp_split, Y_test, Y_pred]
        

Processing.. 0/21
Processing.. 1/21
Processing.. 2/21
Processing.. 3/21
Processing.. 4/21
Processing.. 5/21
Processing.. 6/21
Processing.. 7/21
Processing.. 8/21
Processing.. 9/21
Processing.. 10/21
Processing.. 11/21
Processing.. 12/21
Processing.. 13/21


/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py

Processing.. 14/21


/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):


Processing.. 15/21
Processing.. 16/21


...

Processing.. 17/21


/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):


Processing.. 18/21


/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:113: RuntimeWarning: invalid value encountered in less
  if np.any(dx < 0):
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:651: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py

Processing.. 19/21
Processing.. 20/21


In [13]:
print_results_train, print_results_test = {}, {}

for key_model in cmat_train.keys():
    print_results_train[key_model] = []
    print_results_test[key_model] = []
        
for key_exp in result_dict.keys():
    [cmat_train, cmat_test, auc_test, num_samples, tr_sp, te_sp, Y_test, Y_pred] = result_dict[key_exp]
    for key_model in cmat_train.keys():
        train_prec = cmat_train[key_model][1,1]*1.0/(cmat_train[key_model][1,1] + cmat_train[key_model][2,1])
        test_prec = cmat_test[key_model][1,1]*1.0/(cmat_test[key_model][1,1] + cmat_test[key_model][2,1])
        train_acc = cmat_train[key_model][1:,1:].diagonal()*1.0/cmat_train[key_model][1:,1:].sum(axis=1)
        test_acc = cmat_test[key_model][1:,1:].diagonal()*1.0/cmat_test[key_model][1:,1:].sum(axis=1)
        test_perDN = cmat_train[key_model][1:,0]*1.0/sum(cmat_train[key_model][1:,:], axis=1) # percentage of Don't KNOW
        test_auc = auc_test[key_model]
        f1score_test = 2*(test_acc[0])*test_prec/(test_acc[0] + test_prec)
        print_results_test[key_model].append([key_exp, '%.3f'%mean(test_acc), '%.3f'%test_prec, '%.3f'%test_auc, '%.3f'%mean(f1score_test), '%.3f'%test_acc[0], '%.3f'%test_acc[1], '%.3f'%mean(test_perDN), '%.3f'%test_perDN[0], '%.3f'%test_perDN[1]])
        print_results_train[key_model].append([key_exp, '%.3f'%num_samples, '%.3f'%tr_sp, '%.3f'%te_sp, '%.3f'%train_prec ,'%.3f'%train_acc[0], '%.3f'%train_acc[1]])
    
    
    # Plot error distribution in time
    if plt_err_dist and flag_test:
        decision = (Y_pred == Y_test)
        x = [ind_x for ind_x in range(len(Y_test))]
        x = np.array(x)
        correct_idx = np.argwhere(decision==True)
        incorrect_idx = np.argwhere(decision==False)
        plt.plot(x[correct_idx],Y_test[correct_idx],'b.',label='Correct')
        plt.plot(x[incorrect_idx],Y_test[incorrect_idx],'r.',label='Incorrect')
        plt.xlabel('timesteps')
        yticks(np.array([-5, 0, 1, 5]), ('','ErrP', 'Non-Errp',''))
        plt.legend()
        vert_line = 100
        while vert_line < len(Y_test):
            plt.axvline(x=vert_line)
            vert_line = vert_line + 100
        plt.title(key)
        plt.show()

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
    
print(color.BLUE + "TEST" + color.END)
# print()
# for key_model in print_results_test.keys():        
#     print color.BOLD + key_model.upper() + color.END
#     print tabulate(list(print_results_test[key_model]), headers=['Name', 'ErrP Prec', 'ErrP Acc', 'NonErrP Acc', 'Avg. %DN', '%DN (from ErrP)', '%DN (from NonErrP)'])
#     print ""

print("Name, Avg Accuracy, ErrP Precision, AUC Score, F1, ErrP Acc, NonErrP Acc, Avg DN, ErrP DN, NonErrP DN")
# print()
for key_model in print_results_test.keys():  
    print(color.BOLD + key_model.upper() + color.END)
    for key_sub in print_results_test[key_model]:
        for item in key_sub:
            print("{}, ".format(item),end = '')
        print("")
    
# print()
print(color.RED + "TRAIN" + color.END)
# print()
for key_model in print_results_train.keys():
    print(color.BOLD + key_model.upper() + color.END)
    print(tabulate(list(print_results_train[key_model]), headers=['Name', '#Samples  ', 'Tr_split  ', 'Te_split  ', 'Train ErrP Prec  ', 'Train ErrP Acc  ', 'Train Non_ErrP Acc  ']))
#     print()
    
# print color.RED + "TRAIN" + color.END  
# print ""
# for key_model in print_results_test.keys():
#     print color.BOLD + key_model.upper() + color.END
#     print tabulate(list(print_results_train[key_model]), headers=['Name', '#Samples  ', 'Tr_split  ', 'Te_split  ', 'Train ErrP Prec  ', 'Train ErrP Acc  ', 'Train Non_ErrP Acc  '])
#     print ""


    
    
# Confusion matrix format
#      PRED-- ErrP - Non-ErrP
# TRUE     --------------
# ErrP     --  TP -- FN
# NonErrP  --  FP -- TN)

TEST
Name, Avg Accuracy, ErrP Precision, AUC Score, F1, ErrP Acc, NonErrP Acc, Avg DN, ErrP DN, NonErrP DN
ENSEMBLE
data_S15-0909, 0.881, 0.878, 0.909, 0.832, 0.790, 0.972, 0.154, 0.153, 0.155, 
data_S07-0729, 0.918, 0.947, 0.926, 0.893, 0.845, 0.990, 0.131, 0.125, 0.138, 
data_S01-0717, 0.771, 0.882, 0.824, 0.681, 0.555, 0.987, 0.132, 0.124, 0.140, 
data_S07-0718, 0.922, 0.946, 0.928, 0.905, 0.867, 0.976, 0.185, 0.190, 0.179, 
data_S02-0717, 0.900, 0.902, 0.838, 0.881, 0.861, 0.938, 0.443, 0.452, 0.435, 
data_S01-0719, 0.910, 0.902, 0.909, 0.886, 0.871, 0.949, 0.217, 0.226, 0.208, 
data_S16-0910, 0.921, 0.866, 0.940, 0.868, 0.869, 0.972, 0.121, 0.104, 0.139, 
data_S06-0801, 0.873, 0.822, 0.890, 0.799, 0.778, 0.969, 0.194, 0.174, 0.213, 
data_S01-0629, 0.885, 0.865, 0.902, 0.832, 0.801, 0.969, 0.181, 0.176, 0.185, 
data_S02-0716, 0.818, 0.797, 0.848, 0.730, 0.673, 0.963, 0.212, 0.211, 0.212, 
data_S03-0628, 0.750, 0.657, 0.774, 0.602, 0.556, 0.944, 0.345, 0.355, 0.335, 
data_S07-0626, 